# Python для анализа данных

## Веб-скрэйпинг: скачивание файлов

In [ ]:
#!wget

## Скачивание файлов

Еще одно применение скрэйпинга, о котором мы пока не поговорили - это автоматизированное скачивание файлов. Например, вы хотите скачать научные статьи по определенному ключевому слову или прайс-листы поставщика, которые он загружает на сайт в эскеле.

Давайте посмотрим, как скачивать файлы на примере pdf и заодно попробуем походить по ссылкам. Кстати, этот процесс еще часто называется spidering или crawling, потому что ваш скрипт как паучок ползет по ссылкам (отсюда и название первых поисковых роботов - spider).

Давайте попробуем сохранить англоязычные summary дисертаций, защищенных в 2019 году

Мы уже отредактировали фильтры и ссылка их запомнила. Позже сегодня посмотрим как можно самим заполнять такие поля с помощью Selenium.

https://www.hse.ru/sci/diss/?author=&chief=&year=2019&type=1&degree_type=&council=&spec=&fulltext=yes

In [1]:
!pip install selenium

In [2]:
!pip install webdriver_manager

In [3]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from time import sleep
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
s = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)

driver.implicitly_wait(2)

In [4]:
link = 'https://www.hse.ru/sci/diss/?author=&chief=&year=2019&type=1&degree_type=&council=&spec=&fulltext=yes'
driver.get(link)

In [5]:
soup = BeautifulSoup(driver.page_source)

Давайте для начала поэкспериментируем с первым кандидатом.

In [ ]:
print(soup.prettify())

In [7]:
print(len(soup.find_all('a')))
soup.find_all('a', {'class':'link'})[:50]

174


[<a class="link" href="/sci/diss/314832967">Мотивационные и личностные предпосылки жизненного призвания</a>,
 <a class="link" href="/org/persons/28125914" target="_blank">Леонтьев Дмитрий Алексеевич</a>,
 <a class="link" href="/sci/diss/?keyword=calling">calling</a>,
 <a class="link" href="/sci/diss/?keyword=career%20guidance">career guidance</a>,
 <a class="link" href="/sci/diss/?keyword=meaning%20in%20life">meaning in life</a>,
 <a class="link" href="/sci/diss/?keyword=positive%20psychology%20intervention">positive psychology intervention</a>,
 <a class="link" href="/sci/diss/?keyword=vocation">vocation</a>,
 <a class="link" href="/sci/diss/?keyword=%D0%BF%D0%BE%D0%B7%D0%B8%D1%82%D0%B8%D0%B2%D0%BD%D0%B0%D1%8F%20%D0%B8%D0%BD%D1%82%D0%B5%D1%80%D0%B2%D0%B5%D0%BD%D1%86%D0%B8%D1%8F">позитивная интервенция</a>,
 <a class="link" href="/sci/diss/?keyword=%D0%BF%D1%80%D0%B8%D0%B7%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5">призвание</a>,
 <a class="link" href="/sci/diss/?keyword=%D1%81%D0%BC%D1%8B%D1%81%D

Видим, что ссылок очень много, а нам нужны только те, которые ведут на summary. Можно поискать их по тексту ссылки.

In [8]:
for link in soup.find_all('a', text='Summary'):
    print(link.get('href'))

http://www.hse.ru/data/2019/10/29/1532361365/summary_EN_final (3).pdf
http://www.hse.ru/data/2019/10/22/1528885739/Mitrofanova - summary.pdf
http://www.hse.ru/data/2019/10/18/1543364021/Borodina E._Summary_181019.pdf
http://www.hse.ru/data/2019/10/16/1239735382/Меликян_Summary 16.10.pdf
http://www.hse.ru/data/2019/10/03/1182240389/Кучин_резюме_ENG.pdf
http://www.hse.ru/data/2019/10/15/1539883570/summary.pdf
http://www.hse.ru/data/2019/10/11/1541034023/Resume.pdf
http://www.hse.ru/data/2019/10/11/1535827784/kulikova_thesis summary_final.pdf
http://www.hse.ru/data/2019/08/26/1536057698/Сидорова_summary.pdf
http://www.hse.ru/data/2019/10/04/1184372726/Резюме на английском_Захарова_04.10.2019_финал.pdf


C:\Temp/ipykernel_13440/2000331977.py:1: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  for link in soup.find_all('a', text='Summary'):


И соберем ссылки в список.

In [9]:
links = []
for link in soup.find_all('a', text='Summary'):
#     print(link)
    links.append(link.get('href'))

print(links)

['http://www.hse.ru/data/2019/10/29/1532361365/summary_EN_final (3).pdf', 'http://www.hse.ru/data/2019/10/22/1528885739/Mitrofanova - summary.pdf', 'http://www.hse.ru/data/2019/10/18/1543364021/Borodina E._Summary_181019.pdf', 'http://www.hse.ru/data/2019/10/16/1239735382/Меликян_Summary 16.10.pdf', 'http://www.hse.ru/data/2019/10/03/1182240389/Кучин_резюме_ENG.pdf', 'http://www.hse.ru/data/2019/10/15/1539883570/summary.pdf', 'http://www.hse.ru/data/2019/10/11/1541034023/Resume.pdf', 'http://www.hse.ru/data/2019/10/11/1535827784/kulikova_thesis summary_final.pdf', 'http://www.hse.ru/data/2019/08/26/1536057698/Сидорова_summary.pdf', 'http://www.hse.ru/data/2019/10/04/1184372726/Резюме на английском_Захарова_04.10.2019_финал.pdf']


C:\Temp/ipykernel_13440/621886780.py:2: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  for link in soup.find_all('a', text='Summary'):


Отлично. Теперь нужно придумать, откуда взять название для каждого файла. Пусть это будут фамилии авторов, давайте доберемся до них. Такую задачу мы пока не решали: будем искать тэг по тексту, а потом искать его родителя (потому что ни ячейку таблицы с именем автора, ни саму таблицу не получится уникально отсечь).

In [10]:
for author in soup.find_all('div', {'class':'p1 v'}):
    try:
        print(author.span.text)
    except:
        pass

Белобородова Полина Михайловна
calling, 
анализ наступления событий, 
Бородина Елена Николаевна
вахтовый метод организации труда, 
иностранные студенты, 
модели ценообразования активов, 
магнетронное распыление, 
Годин Андрей Сергеевич
Климов Константин Николаевич
измерения диаграмм направленностей электрически малых антенн, 
NorBA, 
Сидорова Мария Алексеевна
Арендт, 
Захарова Олеся Викторовна
аргументационные схемы (топосы), 


In [11]:
soup.find_all('div', string='Соискатель:')[1].parent()[1].text

'Митрофанова Екатерина Сергеевна'

Достанем фамилии.

In [12]:
authors = []
for author in soup.find_all('div', string='Соискатель:'):
#     print(author.parent()[1].get_text().split()[0])
    authors.append(author.parent()[1].get_text().split()[0])

print(authors)

['Белобородова', 'Митрофанова', 'Бородина', 'Меликян', 'Кучин', 'Монахов', 'Годин', 'Куликова', 'Сидорова', 'Захарова']


Проверим, что списки действительно одинаковой длины.

In [13]:
len(links) == len(authors)

True

In [14]:
print(links)

['http://www.hse.ru/data/2019/10/29/1532361365/summary_EN_final (3).pdf', 'http://www.hse.ru/data/2019/10/22/1528885739/Mitrofanova - summary.pdf', 'http://www.hse.ru/data/2019/10/18/1543364021/Borodina E._Summary_181019.pdf', 'http://www.hse.ru/data/2019/10/16/1239735382/Меликян_Summary 16.10.pdf', 'http://www.hse.ru/data/2019/10/03/1182240389/Кучин_резюме_ENG.pdf', 'http://www.hse.ru/data/2019/10/15/1539883570/summary.pdf', 'http://www.hse.ru/data/2019/10/11/1541034023/Resume.pdf', 'http://www.hse.ru/data/2019/10/11/1535827784/kulikova_thesis summary_final.pdf', 'http://www.hse.ru/data/2019/08/26/1536057698/Сидорова_summary.pdf', 'http://www.hse.ru/data/2019/10/04/1184372726/Резюме на английском_Захарова_04.10.2019_финал.pdf']


Теперь попробуем сохранить файл. У нас все файлы pdf, будем переименовывать их фамилиями автора. Кстати, если файлы разного формата, то расширение можно узнать через атрибут headers

In [15]:
requests.get(links[0]).headers['content-type']

'application/pdf'

In [16]:
# потоковое чтение файла, потому pdf может быть большим и не уместиться в памяти
summary = requests.get(links[0], stream=True)

# на всякий случай делаем проверку, иначе получим битый файл
if summary.headers['content-type'] == 'application/pdf':

    # wb - запись байтовой информации
    fh = open('test.pdf', 'wb')

    # считываем туда "содержание" файла по ссылке
    fh.write(summary.content)
    fh.close()

In [25]:
import os
os.getcwd()
# os.chdir()

'C:\\Users\\i.kosarev'

Давайте теперь еще добавим имя файла по фамилии.

In [18]:
# потоковое чтение файла, потому pdf может быть большим и не уместиться в памяти
summary = requests.get(links[0], stream=True)

# на всякий случай делаем проверку, иначе получим битый файл
if summary.headers['content-type'] == 'application/pdf':

    # wb - запись байтовой информации
    fh = open(f'{authors[0]}.pdf', 'wb')

    # считываем туда "содержание" файла по ссылке
    fh.write(summary.content)
    fh.close()

Давайте сохраним обработку файла в функцию и соберем уже все в цикл.

In [19]:
authors

['Белобородова',
 'Митрофанова',
 'Бородина',
 'Меликян',
 'Кучин',
 'Монахов',
 'Годин',
 'Куликова',
 'Сидорова',
 'Захарова']

In [23]:
import re
def get_pdf(idx):
    summary = requests.get(links[idx], stream=True) # потоковое чтение файла, потому pdf может быть большим и не уместиться в памяти
    if summary.headers['content-type'] == 'application/pdf': # на всякий случай делаем проверку, иначе получим битый файл
        fh = open(f'{authors[idx]}.pdf', 'wb') # wb - запись байтовой информации
        fh.write(summary.content) # считываем туда "содержание" файла по ссылке
        fh.close()

In [ ]:
link = 'https://www.hse.ru/sci/diss/?author=&chief=&year=2019&type=1&degree_type=&council=&spec=&fulltext=yes'
driver.get(link)

soup = BeautifulSoup(driver.page_source)

links = []
for link in soup.find_all('a', text='Summary'):
    links.append(link.get('href'))

authors = []
for author in soup.find_all('div', text='Соискатель:'):
    authors.append(author.parent()[1].get_text().split()[0])

C:\Temp/ipykernel_7876/670446889.py:7: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  for link in soup.find_all('a', text='Summary'):
C:\Temp/ipykernel_7876/670446889.py:11: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  for author in soup.find_all('div', text='Соискатель:'):


In [20]:
len(links)

10

In [21]:
len(authors)

10

In [24]:
for idx in range(len(links)):
    get_pdf(idx)

Готово!